### Project Description
Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster.

###### Submissions are evaluated using F1 between the predicted and expected answers.

## Important Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score,classification_report,confusion_matrix,accuracy_score
import nltk
import re
import textblob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amany\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
tweet_train_data = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\nlp-getting-started\train.csv')
tweet_test_data = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\nlp-getting-started\test.csv')
tweet_train_data.shape,tweet_test_data.shape

((7613, 5), (3263, 4))

In [3]:
tweet_train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
tweet_train_data.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

## Selecting Imp Features

In [5]:
trg_data=tweet_train_data[['text','target']]
test_data=tweet_test_data[['text']]
trg_data.shape,test_data.shape

((7613, 2), (3263, 1))

In [6]:
trg_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
trg_data.text[1]

'Forest fire near La Ronge Sask. Canada'

## Data Filtering

In [8]:
lem=WordNetLemmatizer()

new_data=[]
for sent in range(0,len(trg_data['text'])):
    data=re.sub('[^a-zA-Z0-9]',' ',trg_data.text[sent])
    data=data.lower()
    data=data.split()
    data=[lem.lemmatize(word) for word in data if word not in set(stopwords.words('english'))]
    data=' '.join(data)
    new_data.append(data)

In [9]:
trg_data['text']=pd.DataFrame(new_data,columns=['text'])
trg_data.shape

F:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(7613, 2)

In [10]:
trg_data.head()

,text,target
0,deed reason earthquake may allah forgive u,1
1,forest fire near la ronge sask canada,1
2,resident asked shelter place notified officer ...,1
3,13 000 people receive wildfire evacuation orde...,1
4,got sent photo ruby alaska smoke wildfire pour...,1


## Applying Bag Of words

In [11]:
cv=CountVectorizer(max_features=10000)
text_data=cv.fit_transform(trg_data['text']).toarray()

In [12]:
data_text=pd.DataFrame(text_data)
data_text.shape

(7613, 10000)

In [13]:
data_text['target']=trg_data['target']
data_text['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [14]:
trg_data=data_text.copy()
trg_data.head()

,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Data Splitting

In [15]:
X=trg_data.drop('target',axis=1)
y=trg_data['target']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
x_train.shape,x_test.shape

((5709, 10000), (1904, 10000))

## Model Building

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB(alpha=0.9)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(accuracy_score(y_test,y_pred),'\n')
print(f1_score(y_test,y_pred),'\n')
print(confusion_matrix(y_test,y_pred),'\n')
print(classification_report(y_test,y_pred),'\n')

##It perfomerd well than RandomForestClassifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(accuracy_score(y_test,y_pred),'\n')
print(f1_score(y_test,y_pred),'\n')
print(confusion_matrix(y_test,y_pred),'\n')
print(classification_report(y_test,y_pred),'\n')

0.7967436974789915 

0.7257264351523741 

[[1005  102]
 [ 285  512]] 

              precision    recall  f1-score   support

           0       0.78      0.91      0.84      1107
           1       0.83      0.64      0.73       797

    accuracy                           0.80      1904
   macro avg       0.81      0.78      0.78      1904
weighted avg       0.80      0.80      0.79      1904
 



## HyperParameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid={'n_estimators':[100,120,125,130,135,140,150],
           'n_jobs':[-2,-1,1,2],
           'criterion':["gini","entropy"],
           'class_weight':["balanced", "balanced_subsample"],
           'verbose':[0,1,2,3]
           }
grid_model=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)
grid_model.fit(x_train,y_train)
grid_model.best_params_

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   13.7s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   13.8s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-2)]:

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   13.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   13.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]:

building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


building tree 2 of 125


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s


building tree 3 of 125
building tree 4 of 125
building tree 5 of 125
building tree 6 of 125
building tree 7 of 125
building tree 8 of 125
building tree 9 of 125
building tree 10 of 125
building tree 11 of 125
building tree 12 of 125
building tree 13 of 125
building tree 14 of 125
building tree 15 of 125
building tree 16 of 125
building tree 17 of 125
building tree 18 of 125
building tree 19 of 125
building tree 20 of 125
building tree 21 of 125
building tree 22 of 125
building tree 23 of 125
building tree 24 of 125
building tree 25 of 125
building tree 26 of 125
building tree 27 of 125
building tree 28 of 125
building tree 29 of 125
building tree 30 of 125
building tree 31 of 125
building tree 32 of 125
building tree 33 of 125
building tree 34 of 125
building tree 35 of 125
building tree 36 of 125
building tree 37 of 125
building tree 38 of 125
building tree 39 of 125
building tree 40 of 125
building tree 41 of 125
building tree 42 of 125
building tree 43 of 125
building tree 44 of 125

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    0.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   17.4s
[Parallel(n_jobs=2)]: Done 125 out of 125 | elapsed:   45.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 125 out of 125 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   17.4s
[Parallel(n_jobs=2)]: Done 125 out of 125 | elapsed:   46.8s finis

## Model Builing with Tuned Parameters

In [ ]:
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(accuracy_score(y_test,y_pred),'\n')
print(f1_score(y_test,y_pred),'\n')
print(confusion_matrix(y_test,y_pred),'\n')
print(classification_report(y_test,y_pred),'\n')